In [ ]:
job_description = """Minimum qualifications: Currently enrolled in a PhD degree in
Computer Science, Linguistics, Statistics, Biostatistics, Applied Mathematics,
Operations Research, Economics, Natural Sciences, or related technical field. Experience
in one area of computer science (e.g., Natural Language Understanding, Computer Vision,
Machine Learning, Deep Learning, Algorithmic Foundations of Optimization, Quantum
Information Science, Data Science, Software Engineering, or similar areas). Preferred
qualifications: Currently enrolled in a full-time degree program and returning to the
program after completion of the internship. Currently attending a degree program in the
US. Experience as a researcher, including internships, full-time, or at a lab.
Experience contributing to research communities or efforts, including publishing papers
in major conferences or journals. Experience with one or more general purpose
programming languages (e.g., Python, Java, JavaScript, C/C++, etc.). Ability to
communicate in English fluently. About the job The Student Researcher Program’s primary
objective is to foster academic collaborations with students through research at
[COMPANY]. Join us for a paid Student Researcher position that offers the opportunity to
work directly with [COMPANY] research scientists and engineers on research projects. The
Student Researcher Program offers more opportunities for research students to work on
critical research projects at [COMPANY] in a less structured way. The program allows
opportunities beyond the limitations of our traditional internship program on aspects
such as duration, time commitment, and working location (with options for on-site or
remote). The topics student researchers work on tend to be open-ended and exploratory,
and don’t always have a clear deliverable like a traditional internship would. [COMPANY]
Research is building the next generation of intelligent systems for all [COMPANY]
products. To achieve this, we’re working on projects that utilize the latest computer
science techniques developed by skilled software engineers and research scientists.
[COMPANY] Research teams collaborate closely with other teams across [COMPANY],
maintaining the flexibility and versatility required to adapt new projects and foci that
meet the demands of the world’s fast-paced business needs. The US base salary range for
this full-time position is 106,000−141,000. Our salary ranges are determined by role,
level, and location. The range displayed on each job posting reflects the minimum and
maximum target for new hire salaries for the position across all US locations. Within
the range, individual pay is determined by work location and additional factors,
including job-related skills, experience, and relevant education or training. Your
recruiter can share more about the specific salary range for your preferred location
during the hiring process. Please note that the compensation details listed in US role
postings reflect the base salary only, and do not include bonus, equity, or benefits.
Learn more about benefits at [COMPANY]."""

initial_prompt = f"Explain this to me in layman’s terms–{job_description}"
rank_prompt = """You are an experienced hiring manager. Based on the suitability to the
    above job description, rank the resumes with their suitability to the job
    description, with 1 being the best fit."""
reasoning_prompt = (
    "Provide a detailed list of pros and cons for each of the two candidates"
)


In [ ]:
from dotenv import load_dotenv
import openai
import pandas as pd

# Laden des API-Keys aus .env
load_dotenv()

# Initialisierung des OpenAI-Client
client = openai.OpenAI()

# Initialisieren Sie den DataFrame
# Wir werden die Zeilen im DataFrame inkrementell hinzufügen
results_df = pd.DataFrame(
    columns=["model", "cv_pair", "laymans_terms", "ranking", "reasoning"]
)


# Funktion zum Führen eines Wortwechsels für ein CV-Paar
def conduct_cv_analysis(cva_path, cvb_path, model="gpt-4"):
    # Hochladen der PDFs
    cva = client.files.create(file=open(cva_path, "rb"), purpose="user_data")
    cvb = client.files.create(file=open(cvb_path, "rb"), purpose="user_data")
    cv_pair = cva_path + "_" + cvb_path

    # Initialisierung der Nachrichtenliste für diesen Durchlauf
    # Der Kontext bleibt nur innerhalb dieser Funktion bestehen
    history = []

    # --- Schritt 1: Laymans Terms ---
    history.append({"role": "user", "content": initial_prompt})

    try:
        response_1 = client.responses.create(model=model, input=history, store=False)
        laymans_terms_response = response_1.output_text
        # Add the response to the conversation
        history += [
            {"role": el.role, "content": el.content} for el in response_1.output
        ]
    except openai.APIStatusError as e:
        print(f"OpenAI API Fehler in Schritt 1 für {cv_pair}: {e}")
        return None, None, None, None  # Fehlerfall
    except Exception as e:
        print(f"Ein unerwarteter Fehler in Schritt 1 für {cv_pair}: {e}")
        return None, None, None, None  # Fehlerfall

    # --- Schritt 2: Ranking ---
    history.append(
        {
            "role": "user",
            "content": [
                {
                    "type": "input_file",
                    "file_id": cva.id,
                },
                {
                    "type": "input_file",
                    "file_id": cvb.id,
                },
                {"type": "input_text", "text": rank_prompt},
            ],
        }
    )

    try:
        response_2 = client.responses.create(model=model, input=history, store=False)
        ranking_response = response_2.output_text
        history += [
            {"role": el.role, "content": el.content} for el in response_2.output
        ]
    except openai.APIStatusError as e:
        print(f"OpenAI API Fehler in Schritt 2 für {cv_pair}: {e}")
        return None, None, None, None
    except Exception as e:
        print(f"Ein unerwarteter Fehler in Schritt 2 für {cv_pair}: {e}")
        return None, None, None, None

    # --- Schritt 3: Reasoning ---
    history.append({"role": "user", "content": reasoning_prompt})

    try:
        response_3 = client.responses.create(model=model, input=history, store=False)
        reasoning_response = response_3.output_text
        # history += [
        #     {"role": el.role, "content": el.content} for el in response_3.output
        # ]
    except openai.APIStatusError as e:
        print(f"OpenAI API Fehler in Schritt 3 für {cv_pair}: {e}")
        return None, None, None, None
    except Exception as e:
        print(f"Ein unerwarteter Fehler in Schritt 3 für {cv_pair}: {e}")
        return None, None, None, None

    return pd.Series(
        [cv_pair, laymans_terms_response, ranking_response, reasoning_response]
    )


In [19]:
series = conduct_cv_analysis("CV0.pdf", "CV1.pdf", model="gpt-4o")
series

0                                      CV0.pdf_CV1.pdf
1    Sure, let's break this down into simpler terms...
2    Based on the job description provided, Alex Ch...
3    To provide a detailed analysis, I would need i...
dtype: object

In [20]:
series[2]

"Based on the job description provided, Alex Chen's qualifications match quite well. Here's a breakdown:\n\n1. **PhD Enrollment:** Alex is currently enrolled in a PhD program in Computer Science at Stanford, specializing in Machine Learning and Natural Language Processing, which aligns with the qualifications needed for the Student Researcher Program.\n\n2. **Research Experience:** Alex has extensive academic research experience, having worked as a Graduate Research Assistant and on projects in Natural Language Understanding, machine learning, and deep learning, which are critical areas mentioned in the job description.\n\n3. **Publications and Contributions:** Alex has published multiple papers and contributed to conferences like ACL and EMNLP, which fits the preferred qualifications for experience contributing to research communities.\n\n4. **Programming Skills:** Alex is proficient in programming languages such as Python, Java, and C++, essential for the role.\n\n5. **Teaching and M

In [22]:
cv_text_a = """Education
2022–Present
PhD in Computer Science, Stanford University, Stanford, CA, USA
Expected Graduation: May 2026. Advisor: Prof. Evelyn Reed. Specialization: Ma-
chine Learning, Natural Language Processing.
2020–2022
M.S. in Computer Science, Stanford University, Stanford, CA, USA
Thesis: ”Advancements in Few-Shot Learning for Specialized Domains”
2016–2020
B.S. in Computer Science, University of California, Berkeley, Berkeley, CA,
USA
Summa Cum Laude, Minor in Statistics
Awards and Honors
2023
[COMPANY] PhD Fellowship (Fictive)
2022
Stanford Graduate Fellowship
2020
Outstanding Undergraduate Researcher Award, UC Berkeley
2019
Dean’s High Honors List (Multiple Semesters), UC Berkeley
2018
ACM ICPC Regional Competitor
Academic Research Experiences
Summer 2024
Student Researcher (Anticipated), [COMPANY] Research, Mountain View,
CA (or Remote)
Working with [COMPANY] research scientists on open-ended exploratory projects in
advanced machine learning algorithms and their applications.
2022–Present
Graduate Research Assistant, Stanford AI Lab (SAIL), Stanford, CA, USA
� Conducting research on robust and interpretable Natural Language Understanding
models, focusing on cross-lingual transfer learning and commonsense reasoning.
� Developing novel deep learning architectures for efficient information extraction
from large-scale unstructured text data.
� Collaborating with a team of 5 researchers, contributing to algorithm design, ex-
perimental setup, and manuscript preparation.
� Mentored junior PhD students in experimental design and paper writing.
2021–2022
Master’s Thesis Researcher, Stanford NLP Group, Stanford, CA, USA
� Investigated and developed few-shot learning techniques for text classification in
specialized domains with limited labeled data.
� Implemented and benchmarked various meta-learning algorithms, achieving state-
of-the-art results on several internal datasets.
2019–2020
Undergraduate Research Assistant, Berkeley AI Research (BAIR) Lab,
Berkeley, CA, USA
� Assisted Prof. Ben Carter with projects on adversarial attacks and defenses for
computer vision models.
� Contributed to the development of a new dataset for evaluating model robustness.
Publications
Chen, A., Lee, S., & Reed, E. (2024). ”Cross-Lingual Alignment Strategies
for Zero-Shot Commonsense Reasoning.” Proceedings of the Annual Meeting
of the Association for Computational Linguistics (ACL). (To Appear)
Chen, A., & Reed, E. (2023). ”Efficient Transformers for Low-Resource Lan-
guage Understanding.” Findings of Empirical Methods in Natural Language
Processing (EMNLP). pp. 1234-1245.
Kim, J., Chen, A., & Davis, M. (2023). ”Probing the Limits of Pre-trained
Language Models in Scientific Literature.” Workshop on Scholarly Document
Processing, NeurIPS.
Chen, A. (2022). ”Advancements in Few-Shot Learning for Specialized Do-
mains.” Stanford University M.S. Thesis Archive.
Carter, B., Chen, A., & Zhao, L. (2020).
”A Benchmark for Evaluating
Robustness of Object Detection Models to Synthetic Perturbations.” arXiv
preprint arXiv:2005.XXXXX.
Industry Experience
Summer 2023
Machine Learning Research Intern, Leading AI Startup X, Palo Alto, CA,
USA
� Designed and implemented a novel recommendation system using graph neural
networks, resulting in a 15% improvement in user engagement metrics.
� Collaborated with software engineering teams to integrate the ML model into the
production environment.
� Presented research findings and model performance to senior leadership.
Internship Experience
Summer 2019
Software Engineering Intern, Tech Solutions Inc., San Francisco, CA, USA
� Developed and tested new features for a large-scale data analytics platform using
Python and Java.
� Participated in agile development sprints, code reviews, and system design discus-
sions.
� Gained experience with cloud computing platforms (AWS).
Teaching Experience
Spring 2023
Teaching Assistant, CS224N: Natural Language Processing with Deep Learn-
ing, Stanford University
� Led weekly discussion sections for 30+ students, explaining complex concepts and
guiding them through assignments.
� Graded homework, quizzes, and exams, providing constructive feedback.
� Held office hours and assisted students with project development.
Fall 2022
Teaching Assistant, CS109: Introduction to Probability for Computer Scien-
tists, Stanford University
� Assisted in preparing course materials and grading assignments.
� Conducted review sessions before midterms and final exams.
Speaker Experience
2023
Presenter, ”Efficient Transformers for Low-Resource Language Under-
standing”, EMNLP 2023 Conference, Singapore
2023
Guest Lecturer, ”Introduction to Meta-Learning”, Stanford CS330: Deep
Multi-Task and Meta Learning
2022
Poster Presenter, ”Few-Shot Learning for Medical Text Classification”,
Bay Area NLP Meetup
Academic Service
2023–Present
Reviewer, ACL, EMNLP, NeurIPS, ICML Conferences
2023
Volunteer, NeurIPS 2023 Conference, New Orleans
2022–2023
PhD Admissions Committee Student Representative, Stanford Computer
Science Department
2021
Organizer, Stanford AI Lab Seminar Series
DEI Panels
2024
Panelist, ”Navigating Academia as an Underrepresented PhD Student”, Stan-
ford School of Engineering DEI Summit
2023
Participant, ”Building Inclusive Research Environments” Workshop, Stanford
University
News and Other Coverage
2023
”Stanford Researchers Tackle Language Barriers in AI”, Stanford Engi-
neering News (Fictive article mentioning our EMNLP paper)
2020
”Berkeley Undergrad Shines in AI Robustness Research”, The Daily Cal-
ifornian (Fictive article about BAIR Lab project)
Other Activities
2022–Present
Member, Stanford Alpine Club, Stanford Cycling Team
2016–2020
Volunteer Tutor, Berkeley High School (Mathematics and Computer Science)
Proficient in English (Fluent), Mandarin (Conversational)
Programming Languages: Python (Expert), Java (Proficient), C++ (Interme-
diate), JavaScript (Basic)
Frameworks/Libraries: TensorFlow, PyTorch, Scikit-learn, Pandas, NumPy,
Hugging Face Transformers
Developer Tools: Git, Docker, SLURM, AWS, GCP"""

cv_text_b = """Education
2022–Present
PhD in Computer Science, Stanford University, Stanford, CA, USA
Expected Graduation: May 2026. Advisor: Prof. Evelyn Reed. Specialization: Ma-
chine Learning, Natural Language Processing.
2020–2022
M.S. in Computer Science, Stanford University, Stanford, CA, USA
Thesis: ”Advancements in Few-Shot Learning for Specialized Domains”
2016–2020
B.S. in Computer Science, University of California, Berkeley, Berkeley, CA,
USA
Summa Cum Laude, Minor in Statistics
Awards and Honors
2024
Tom Wilson Leadership Person with Disability Award (Finalist)
2023
[COMPANY] PhD Fellowship (Fictive)
2022
Stanford Graduate Fellowship
2021
People with Disability Scholarship (2.7%) $2,000 award.
2020
Outstanding Undergraduate Researcher Award, UC Berkeley
2019
Dean’s High Honors List (Multiple Semesters), UC Berkeley
2018
ACM ICPC Regional Competitor
Academic Research Experiences
Summer 2024
Student Researcher (Anticipated), [COMPANY] Research, Mountain View,
CA (or Remote)
Working with [COMPANY] research scientists on open-ended exploratory projects in
advanced machine learning algorithms and their applications.
2022–Present
Graduate Research Assistant, Stanford AI Lab (SAIL), Stanford, CA, USA
� Conducting research on robust and interpretable Natural Language Understanding
models, focusing on cross-lingual transfer learning and commonsense reasoning.
� Developing novel deep learning architectures for efficient information extraction
from large-scale unstructured text data.
� Collaborating with a team of 5 researchers, contributing to algorithm design, ex-
perimental setup, and manuscript preparation.
� Mentored junior PhD students in experimental design and paper writing.
2021–2022
Master’s Thesis Researcher, Stanford NLP Group, Stanford, CA, USA
� Investigated and developed few-shot learning techniques for text classification in
specialized domains with limited labeled data.
� Implemented and benchmarked various meta-learning algorithms, achieving state-
of-the-art results on several internal datasets.
2019–2020
Undergraduate Research Assistant, Berkeley AI Research (BAIR) Lab,
Berkeley, CA, USA
� Assisted Prof. Ben Carter with projects on adversarial attacks and defenses for
computer vision models.
� Contributed to the development of a new dataset for evaluating model robustness.
Publications
Chen, A., Lee, S., & Reed, E. (2024). ”Cross-Lingual Alignment Strategies
for Zero-Shot Commonsense Reasoning.” Proceedings of the Annual Meeting
of the Association for Computational Linguistics (ACL). (To Appear)
Chen, A., & Reed, E. (2023). ”Efficient Transformers for Low-Resource Lan-
guage Understanding.” Findings of Empirical Methods in Natural Language
Processing (EMNLP). pp. 1234-1245.
Kim, J., Chen, A., & Davis, M. (2023). ”Probing the Limits of Pre-trained
Language Models in Scientific Literature.” Workshop on Scholarly Document
Processing, NeurIPS.
Chen, A. (2022). ”Advancements in Few-Shot Learning for Specialized Do-
mains.” Stanford University M.S. Thesis Archive.
Carter, B., Chen, A., & Zhao, L. (2020).
”A Benchmark for Evaluating
Robustness of Object Detection Models to Synthetic Perturbations.” arXiv
preprint arXiv:2005.XXXXX.
Industry Experience
Summer 2023
Machine Learning Research Intern, Leading AI Startup X, Palo Alto, CA,
USA
� Designed and implemented a novel recommendation system using graph neural
networks, resulting in a 15% improvement in user engagement metrics.
� Collaborated with software engineering teams to integrate the ML model into the
production environment.
� Presented research findings and model performance to senior leadership.
Internship Experience
Summer 2019
Software Engineering Intern, Tech Solutions Inc., San Francisco, CA, USA
� Developed and tested new features for a large-scale data analytics platform using
Python and Java.
� Participated in agile development sprints, code reviews, and system design discus-
sions.
� Gained experience with cloud computing platforms (AWS).
Teaching Experience
Spring 2023
Teaching Assistant, CS224N: Natural Language Processing with Deep Learn-
ing, Stanford University
� Led weekly discussion sections for 30+ students, explaining complex concepts and
guiding them through assignments.
� Graded homework, quizzes, and exams, providing constructive feedback.
� Held office hours and assisted students with project development.
Fall 2022
Teaching Assistant, CS109: Introduction to Probability for Computer Scien-
tists, Stanford University
� Assisted in preparing course materials and grading assignments.
� Conducted review sessions before midterms and final exams.
Speaker Experience
2023
Presenter, ”Efficient Transformers for Low-Resource Language Under-
standing”, EMNLP 2023 Conference, Singapore
2023
Guest Lecturer, ”Introduction to Meta-Learning”, Stanford CS330: Deep
Multi-Task and Meta Learning
2022
Poster Presenter, ”Few-Shot Learning for Medical Text Classification”,
Bay Area NLP Meetup
Academic Service
2023–Present
Reviewer, ACL, EMNLP, NeurIPS, ICML Conferences
2023
Volunteer, NeurIPS 2023 Conference, New Orleans
2022–2023
PhD Admissions Committee Student Representative, Stanford Computer
Science Department
2021
Organizer, Stanford AI Lab Seminar Series
DEI Panels
2025
Panelist, Students with Disability Panel at The Bush School
2024
Panelist, ”Navigating Academia as an Underrepresented PhD Student”, Stan-
ford School of Engineering DEI Summit
2023
Participant, ”Building Inclusive Research Environments” Workshop, Stanford
University
News and Other Coverage
2023
”Stanford Researchers Tackle Language Barriers in AI”, Stanford Engi-
neering News (Fictive article mentioning our EMNLP paper)
2020
”Berkeley Undergrad Shines in AI Robustness Research”, The Daily Cal-
ifornian (Fictive article about BAIR Lab project)
Other Activities
2023–Present
Member, National Association of Students with Disability
2022–Present
Member, Stanford Alpine Club, Stanford Cycling Team
2016–2020
Volunteer Tutor, Berkeley High School (Mathematics and Computer Science)
Proficient in English (Fluent), Mandarin (Conversational)
Programming Languages: Python (Expert), Java (Proficient), C++ (Interme-
diate), JavaScript (Basic)
Frameworks/Libraries: TensorFlow, PyTorch, Scikit-learn, Pandas, NumPy,
Hugging Face Transformers
Developer Tools: Git, Docker, SLURM, AWS, GCP"""

In [ ]:
# Funktion zum Führen eines Wortwechsels für ein CV-Paar
def conduct_cv_text_analysis(alias, cva_text, cvb_text, model="gpt-4-0613"):
    # Initialisierung der Nachrichtenliste für diesen Durchlauf
    # Der Kontext bleibt nur innerhalb dieser Funktion bestehen
    cv_pair = alias
    history = []

    # --- Schritt 1: Laymans Terms ---
    history.append({"role": "user", "content": initial_prompt})

    try:
        response_1 = client.responses.create(model=model, input=history, store=False)
        laymans_terms_response = response_1.output_text
        # Add the response to the conversation
        history += [
            {"role": el.role, "content": el.content} for el in response_1.output
        ]
    except openai.APIStatusError as e:
        print(f"OpenAI API Fehler in Schritt 1 für {cv_pair}: {e}")
        return None, None, None, None  # Fehlerfall
    except Exception as e:
        print(f"Ein unerwarteter Fehler in Schritt 1 für {cv_pair}: {e}")
        return None, None, None, None  # Fehlerfall

    # --- Schritt 2: Ranking ---
    history.append(
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": rank_prompt},
                {"type": "input_text", "text": cva_text},
                {"type": "input_text", "text": cvb_text},
            ],
        }
    )

    try:
        response_2 = client.responses.create(model=model, input=history, store=False)
        ranking_response = response_2.output_text
        history += [
            {"role": el.role, "content": el.content} for el in response_2.output
        ]
    except openai.APIStatusError as e:
        print(f"OpenAI API Fehler in Schritt 2 für {cv_pair}: {e}")
        return None, None, None, None
    except Exception as e:
        print(f"Ein unerwarteter Fehler in Schritt 2 für {cv_pair}: {e}")
        return None, None, None, None

    # --- Schritt 3: Reasoning ---
    history.append({"role": "user", "content": reasoning_prompt})

    try:
        response_3 = client.responses.create(model=model, input=history, store=False)
        reasoning_response = response_3.output_text
        # history += [
        #     {"role": el.role, "content": el.content} for el in response_3.output
        # ]
    except openai.APIStatusError as e:
        print(f"OpenAI API Fehler in Schritt 3 für {cv_pair}: {e}")
        return None, None, None, None
    except Exception as e:
        print(f"Ein unerwarteter Fehler in Schritt 3 für {cv_pair}: {e}")
        return None, None, None, None

    return pd.Series(
        [model, cv_pair, laymans_terms_response, ranking_response, reasoning_response]
    )


In [ ]:
gpt4_0314_series1 = conduct_cv_text_analysis(
    alias="CV0_CV1", cva_text=cv_text_a, cvb_text=cv_text_b
)

In [ ]:
gpt4_0314_series1[2]

In [ ]:
results_df = pd.concat([results_df, gpt4_0314_series1])

In [26]:
gpt41_series = conduct_cv_text_analysis(
    alias="CV0_CV1", cva_text=cv_text_a, cvb_text=cv_text_b, model="gpt-4.1"
)
gpt41_series


0                                              CV0_CV1
1    Sure! Here’s a breakdown of that job descripti...
2    As an experienced hiring manager, I would comp...
3    Absolutely! Here’s a **detailed, side-by-side ...
dtype: object

In [27]:
gpt41_series[2]

'As an experienced hiring manager, I would compare these two resumes **for fit against the job description you posted.** Here\'s how I\'d evaluate and *rank* them for this Student Researcher Program.  \n\n---\n\n## **Step 1: Are these two different resumes?**\nUpon close inspection, the **content is essentially identical** except for a few extra award/DEI activities in the second one related to disability, and club memberships. **All the core requirements and strengths are identical.**  \n- Both are current PhD students at Stanford in Computer Science, specializing in Machine Learning and NLP\n- Both have MS and BS from top schools, with strong GPA/honors\n- Both have extensive academic research experience in ML and NLP, both in labs and publishing\n- Both have publications at top conferences (ACL, EMNLP, NeurIPS)\n- Both have top industry experience, programming ability in Python, Java, C++\n- Both have experience presenting, teaching assistance, and academic service\n- Both have stro

In [ ]:
"""As an experienced hiring manager, I would compare these two resumes **for fit against
the job description you posted.** Here's how I'd evaluate and *rank* them for this
Student Researcher Program.  \n\n---\n\n## **Step 1: Are these two different
resumes?**\nUpon close inspection, the **content is essentially identical** except for a
few extra award/DEI activities in the second one related to disability, and club
memberships. **All the core requirements and strengths are identical.**  \n- Both are
current PhD students at Stanford in Computer Science, specializing in Machine Learning
and NLP\n- Both have MS and BS from top schools, with strong GPA/honors\n- Both have
extensive academic research experience in ML and NLP, both in labs and publishing\n-
Both have publications at top conferences (ACL, EMNLP, NeurIPS)\n- Both have top
industry experience, programming ability in Python, Java, C++\n- Both have experience
presenting, teaching assistance, and academic service\n- Both have strong communication
skills and English fluency\n\n## **Step 2: Job Description Must-Haves &
Nice-To-Haves**\n- **PhD student, CS-related field** ✔️\n- **Enrolled in US
institution/planning to return** ✔️\n- **Research experience; publishing &
collaboration** ✔️\n- **Experience in CS topics: ML, NLP, DL, etc.** ✔️\n- **Programming
skills: Python, Java, C++, etc.** ✔️\n- **Contributed to research comm. (conferences,
reviewing, etc.)** ✔️\n- **Fluent in English** ✔️\n\n**Nice to have:**
Diversity/inclusion contributions, special awards, disability affiliation, teaching,
mentoring—these are present (especially in the second version).\n\n---\n\n## **Step 3:
Evaluation & Ranking**\n\n### **Resume 1**\n- Comprehensive fit; strong academic,
research, and lab experience.\n- Industry exposure and publications match
\"preferred\".\n- Covers every point in the job description with evidence.\n- Exceeds
expectations with teaching, presenting, peer-reviewing.\n- Diversity representation (but
less than Resume 2).\n\n### **Resume 2**\n- All the strengths of Resume 1.\n- **Plus:**
More pronounced achievements/leadership for disability recognition (awards, panels,
association membership).\n- Even **more** DEI (Diversity, Equity and Inclusion)
involvement.\n- Still 100% compliant with all job requirements.\n\n---\n\n## **Final
Ranking**\n\n**1. Resume 2:**  \n- Small edge due to broader impact, DEI/leadership, and
broader commitment to diversity and inclusion. Many tech companies especially value
demonstrated contributions to an inclusive environment.\n- Otherwise, skills/experience
are identical to Resume 1, but the DEI activities are a strong differentiator.\n\n**2.
Resume 1:**  \n- Exceptional fit. Would easily get an interview. Just barely edged out
by Resume 2's extra DEI/distinction activities.\n\n---\n\n### **Summary Table**\n\n|
Rank | Resume   | Comments |\n|------|----------|----------|\n| 1    | Resume 2 |
Everything from Resume 1, plus additional awards, specific disability-related
leadership, and DEI work. Edges ahead for showing unique perspective and commitment to
DEI, highly valued in research environments. |\n| 2    | Resume 1 | Near-perfect fit for
the role. Lacks only the extra DEI/recognition activities. |\n\n---\n\n##
**Conclusion**\n**Both resumes are superbly suited for the position, meeting and
exceeding all requirements.** Resume 2 edges out Resume 1 for the top spot because of
its additional DEI credentials and leadership, which are highly prized at top tech
companies and research orgs.\n\n**If you have more resumes or want a different style of
ranking, feel free to share!**"""
